# Lab 1: Building own n-gram language model

For this we will start with already tokenized form of data.

In [1]:
# Tokens of the sentences
sample1 = ['It', 'shows', ',', 'my', 'dear', 'Watson', ',', 'that',
           'we', 'are', 'dealing', 'with', 'an', 'exceptionally',
           'astute', 'and', 'dangerous', 'man', '.']
sample2 = ['How', 'would', 'Lausanne', 'do', ',', 'my', 'dear',
           'Watson', '?']

Our first task is to write a function that splits the `tokens` sequence
into its `n`-grams.

For instance, when `tokens=sample1` and `n=3`, your function should
return:

```python
[('It', 'shows', ','),
 ('shows', ',', 'my'),
 (',', 'my', 'dear'),
 ...,
 ('dangerous', 'man', '.')]
```

Note: You should return a python [`list`](https://docs.python.org/3/tutorial/datastructures.html#more-on-lists) containing [`tuple`](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences)s. `tuple`s are immutable sequences, which will be useful later on when you build your language model.

In [2]:
from typing import List, Tuple

def build_ngrams(tokens: List[str], n: int) -> List[Tuple[str]]:
    """
    This function takes a list of tokens and an integer n,
    and returns a list of n-grams (as tuples) from the token list.
    """
    ngrams = []
    
    for i in range(len(tokens) - n+1):
        ngram = tuple(tokens[i:i+n])
        ngrams.append(ngram)
    return ngrams

# Example
build_ngrams(sample1, n=3)            

[('It', 'shows', ','),
 ('shows', ',', 'my'),
 (',', 'my', 'dear'),
 ('my', 'dear', 'Watson'),
 ('dear', 'Watson', ','),
 ('Watson', ',', 'that'),
 (',', 'that', 'we'),
 ('that', 'we', 'are'),
 ('we', 'are', 'dealing'),
 ('are', 'dealing', 'with'),
 ('dealing', 'with', 'an'),
 ('with', 'an', 'exceptionally'),
 ('an', 'exceptionally', 'astute'),
 ('exceptionally', 'astute', 'and'),
 ('astute', 'and', 'dangerous'),
 ('and', 'dangerous', 'man'),
 ('dangerous', 'man', '.')]

In [3]:
# Tests:
assert len(build_ngrams(sample1, n=3)) == 17
assert build_ngrams(sample1, n=3)[0] == ('It', 'shows', ',')
assert build_ngrams(sample1, n=3)[10] == ('dealing', 'with', 'an')
assert len(build_ngrams(sample1, n=2)) == 18
assert build_ngrams(sample1, n=2)[0] == ('It', 'shows')
assert build_ngrams(sample1, n=2)[10] == ('dealing', 'with')
assert len(build_ngrams(sample2, n=2)) == 8
assert build_ngrams(sample2, n=2)[0] == ('How', 'would')
assert build_ngrams(sample2, n=2)[7] == ('Watson', '?')

With the current function, there's no way to know whether an n-gram is at the beginning, middle, or end of the sequence. To overcome this problem, n-gram language models often include special "beginning-of-string" (BOS) and "end-of-string" (EOS) control tokens.

Write a new version of our `build_ngrams` function that includes these control tokens. For instance, when `tokens=sample1` and `n=3`, your new function should return:

```python
[('<BOS>', '<BOS>', 'It'),
 ('<BOS>', 'It', 'shows'),
 ('It', 'shows', ','),
 ('shows', ',', 'my'),
 (',', 'my', 'dear'),
 ...,
 ('dangerous', 'man', '.'),
 ('man', '.', '<EOS>'),
 ('.', '<EOS>', '<EOS>')]
```

In [4]:
BOS = '<BOS>'
EOS = '<EOS>'

def build_ngrams_ctrl(tokens: List[str], n: int) -> List[Tuple[str]]:
    """
    Enhance the build_ngrams function to include beginning-of-sequence (BOS)
    and end-of-sequence (EOS) control tokens.
    """
    
    modified_tokens = [BOS] * (n-1) + tokens + [EOS] * (n-1)
    ngrams = []
    
    for i in range(len(modified_tokens) - n+1):
        ngram = tuple(modified_tokens[i: i+n])
        ngrams.append(ngram)
    return ngrams
    
# Example:
build_ngrams_ctrl(sample1, n=3)

[('<BOS>', '<BOS>', 'It'),
 ('<BOS>', 'It', 'shows'),
 ('It', 'shows', ','),
 ('shows', ',', 'my'),
 (',', 'my', 'dear'),
 ('my', 'dear', 'Watson'),
 ('dear', 'Watson', ','),
 ('Watson', ',', 'that'),
 (',', 'that', 'we'),
 ('that', 'we', 'are'),
 ('we', 'are', 'dealing'),
 ('are', 'dealing', 'with'),
 ('dealing', 'with', 'an'),
 ('with', 'an', 'exceptionally'),
 ('an', 'exceptionally', 'astute'),
 ('exceptionally', 'astute', 'and'),
 ('astute', 'and', 'dangerous'),
 ('and', 'dangerous', 'man'),
 ('dangerous', 'man', '.'),
 ('man', '.', '<EOS>'),
 ('.', '<EOS>', '<EOS>')]

In [5]:
# Tests:
assert len(build_ngrams_ctrl(sample1, n=3)) == 21
assert build_ngrams_ctrl(sample1, n=3)[0] == ('<BOS>', '<BOS>', 'It')
assert build_ngrams_ctrl(sample1, n=3)[10] == ('we', 'are', 'dealing')
assert len(build_ngrams_ctrl(sample1, n=2)) == 20
assert build_ngrams_ctrl(sample1, n=2)[0] == ('<BOS>', 'It')
assert build_ngrams_ctrl(sample1, n=2)[10] == ('are', 'dealing')
assert len(build_ngrams_ctrl(sample2, n=2)) == 10
assert build_ngrams_ctrl(sample2, n=2)[0] == ('<BOS>', 'How')
assert build_ngrams_ctrl(sample2, n=2)[9] == ('?', '<EOS>')

Now that you can build n-grams, we have almost everything we need to build an n-gram language model.

To compute Maximum Likelihood Estimations, you first need to count the number of times each word follows an n-gram of size `n-1`. You can build this structure as a Python [`dict`](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) that maps the n-grams of size `n-1` to another `dict` that maps the following words to their respective counts.

For instance, when `texts=[sample1, sample2]` and `n=3`, your function should return:

```python
{
    ('<BOS>', '<BOS>'): {'It': 1, 'How': 1},
    ('<BOS>', 'It'): {'shows': 1},
    ('<BOS>', 'How'): {'would': 1},
    ...
    ('my', 'dear'): {'Watson': 2},
    ('dear', 'Watson'): {',': 1, '?': 1},
    ...
}
```

In [6]:
from typing import Dict
def count_ngrams(texts: List[List[str]], n: int) -> Dict[Tuple[str, ...], Dict[str, int]]:
    """
    counts the occurrences of each word following an n-gram
    of size n-1 in the given text
    """
    ngram_counts = {}
    
    for tokens in texts:
        ngrams = build_ngrams_ctrl(tokens, n)
        for ngram in ngrams:
            prefix = ngram[:-1]
            following_word = ngram[-1]
            
            if prefix not in ngram_counts:
                ngram_counts[prefix] = {}
            if following_word not in ngram_counts[prefix]:
                ngram_counts[prefix][following_word] = 1
            else:
                ngram_counts[prefix][following_word] +=1
    return ngram_counts

# Example:
count_ngrams([sample1, sample2], n=3)

{('<BOS>', '<BOS>'): {'It': 1, 'How': 1},
 ('<BOS>', 'It'): {'shows': 1},
 ('It', 'shows'): {',': 1},
 ('shows', ','): {'my': 1},
 (',', 'my'): {'dear': 2},
 ('my', 'dear'): {'Watson': 2},
 ('dear', 'Watson'): {',': 1, '?': 1},
 ('Watson', ','): {'that': 1},
 (',', 'that'): {'we': 1},
 ('that', 'we'): {'are': 1},
 ('we', 'are'): {'dealing': 1},
 ('are', 'dealing'): {'with': 1},
 ('dealing', 'with'): {'an': 1},
 ('with', 'an'): {'exceptionally': 1},
 ('an', 'exceptionally'): {'astute': 1},
 ('exceptionally', 'astute'): {'and': 1},
 ('astute', 'and'): {'dangerous': 1},
 ('and', 'dangerous'): {'man': 1},
 ('dangerous', 'man'): {'.': 1},
 ('man', '.'): {'<EOS>': 1},
 ('.', '<EOS>'): {'<EOS>': 1},
 ('<BOS>', 'How'): {'would': 1},
 ('How', 'would'): {'Lausanne': 1},
 ('would', 'Lausanne'): {'do': 1},
 ('Lausanne', 'do'): {',': 1},
 ('do', ','): {'my': 1},
 ('Watson', '?'): {'<EOS>': 1},
 ('?', '<EOS>'): {'<EOS>': 1}}

In [7]:
# Tests:
assert len(count_ngrams([sample1, sample2], n=3)) == 28
assert len(count_ngrams([sample1, sample2], n=3)['<BOS>', '<BOS>']) == 2
assert count_ngrams([sample1, sample2], n=3)['<BOS>', '<BOS>']['It'] == 1
assert count_ngrams([sample1, sample2], n=3)['<BOS>', '<BOS>']['How'] == 1
assert count_ngrams([sample1, sample2], n=3)['my', 'dear']['Watson'] == 2
assert len(count_ngrams([sample1, sample2], n=2)) == 24
assert len(count_ngrams([sample1, sample2], n=2)['<BOS>',]) == 2
assert count_ngrams([sample1, sample2], n=2)['<BOS>',]['It'] == 1
assert count_ngrams([sample1, sample2], n=2)['<BOS>',]['How'] == 1
assert count_ngrams([sample1, sample2], n=2)['dear',]['Watson'] == 2

We're almost there! The last step is to convert the counts into probability estimates.

When `texts=[sample1, sample2]` and `n=3`, your function should return:

```python
{
    ('<BOS>', '<BOS>'): {'It': 0.5, 'How': 0.5},
    ('<BOS>', 'It'): {'shows': 1.0},
    ('<BOS>', 'How'): {'would': 1.0},
    ...
    ('my', 'dear'): {'Watson': 1.0},
    ('dear', 'Watson'): {',': 0.5, '?': 0.5},
    ...
}
```

In [8]:
from typing import Dict
def build_ngram_model(texts: List[List[str]], n: int) -> Dict[Tuple[str, ...], Dict[str, float]]:
    """
    Convert n-gram counts to probability estimates.
    """
    ngram_probabilities = {}
    ngram_counts = count_ngrams(texts, n)
    
    for prefix, following_words in ngram_counts.items():
        total_count = sum(following_words.values())
        ngram_probabilities[prefix] = {word: count / total_count for word, count in following_words.items()}
    
    return ngram_probabilities

# Example:
build_ngram_model([sample1, sample2], n=3)

{('<BOS>', '<BOS>'): {'It': 0.5, 'How': 0.5},
 ('<BOS>', 'It'): {'shows': 1.0},
 ('It', 'shows'): {',': 1.0},
 ('shows', ','): {'my': 1.0},
 (',', 'my'): {'dear': 1.0},
 ('my', 'dear'): {'Watson': 1.0},
 ('dear', 'Watson'): {',': 0.5, '?': 0.5},
 ('Watson', ','): {'that': 1.0},
 (',', 'that'): {'we': 1.0},
 ('that', 'we'): {'are': 1.0},
 ('we', 'are'): {'dealing': 1.0},
 ('are', 'dealing'): {'with': 1.0},
 ('dealing', 'with'): {'an': 1.0},
 ('with', 'an'): {'exceptionally': 1.0},
 ('an', 'exceptionally'): {'astute': 1.0},
 ('exceptionally', 'astute'): {'and': 1.0},
 ('astute', 'and'): {'dangerous': 1.0},
 ('and', 'dangerous'): {'man': 1.0},
 ('dangerous', 'man'): {'.': 1.0},
 ('man', '.'): {'<EOS>': 1.0},
 ('.', '<EOS>'): {'<EOS>': 1.0},
 ('<BOS>', 'How'): {'would': 1.0},
 ('How', 'would'): {'Lausanne': 1.0},
 ('would', 'Lausanne'): {'do': 1.0},
 ('Lausanne', 'do'): {',': 1.0},
 ('do', ','): {'my': 1.0},
 ('Watson', '?'): {'<EOS>': 1.0},
 ('?', '<EOS>'): {'<EOS>': 1.0}}

In [9]:
# Tests:
assert build_ngram_model([sample1, sample2], n=3)['<BOS>', '<BOS>']['It'] == 0.5
assert build_ngram_model([sample1, sample2], n=3)['<BOS>', '<BOS>']['How'] == 0.5
assert build_ngram_model([sample1, sample2], n=3)['my', 'dear']['Watson'] == 1.0
assert build_ngram_model([sample1, sample2], n=2)['<BOS>',]['It'] == 0.5
assert build_ngram_model([sample1, sample2], n=2)['<BOS>',]['How'] == 0.5
assert build_ngram_model([sample1, sample2], n=2)['dear',]['Watson'] == 1.0

A language model built from only a few sentences is not very informative. Let's scale up and see what your language model looks like when we train on the complete works of Sir Arthur Conon Doyle!

In [10]:
full_text = []
with open('arthur-conan-doyle.tok.train.txt', 'rt') as fin:
    for line in fin:
        full_text.append(list(line.split()))
        
model = build_ngram_model(full_text, n=3)

In [11]:
for prefix in [(BOS, BOS), (BOS, 'It'), ('It', 'was'), ('my', 'dear')]:
    print(*prefix)
    sorted_probs = sorted(model[prefix].items(), key=lambda x: -x[1])
    for k, v in sorted_probs[:5]:
        print(f'\t{k}\t{v:.4f}')
    print(f'\t[{len(sorted_probs)-5} more...]')

<BOS> <BOS>
	"	0.8073
	The	0.0304
	Holmes	0.0230
	I	0.0167
	It	0.0141
	[206 more...]
<BOS> It
	was	0.8235
	is	0.0515
	may	0.0221
	had	0.0147
	seemed	0.0074
	[11 more...]
It was
	a	0.1888
	the	0.0686
	not	0.0562
	only	0.0312
	an	0.0250
	[184 more...]
my dear
	Watson	0.5612
	fellow	0.1429
	sir	0.0918
	young	0.0510
	Von	0.0204
	[12 more...]


In [12]:
import random

def generate_sentence(model, n=3, start_tokens=('<BOS>',), stop_token='<EOS>', max_length=20):
    current_prefix = tuple(start_tokens) * (n-1)  # Adjust for n-gram size
    sentence = list(current_prefix[:-1])  # Initialize sentence
    
    for _ in range(max_length):
        next_words = model.get(current_prefix)
        if not next_words:
            break  # Stop if the prefix is not found
        
        # Choose the next word based on the probabilities
        next_word = random.choices(list(next_words.keys()), weights=next_words.values())[0]
        if next_word == stop_token:
            break  # Stop if the end-of-sequence token is reached
        
        sentence.append(next_word)
        # Update the prefix
        current_prefix = (*current_prefix[1:], next_word)
    
    return ' '.join(sentence)

# Example usage
BOS = '<BOS>'
EOS = '<EOS>'
# Assuming `model` is your trained n-gram model
sentence = generate_sentence(model, n=3, start_tokens=(BOS,), stop_token=EOS)
print(sentence)


<BOS> As I glanced at each of them which he handed it to the south of France . This time his


In [16]:
import random

def generate_from_ngram(model, start_prefix, num_words=50):
    """
    Generate words based on a given start prefix using the n-gram model.
    
    Parameters:
        model (dict): The n-gram model as a nested dictionary.
        start_prefix (tuple): The starting prefix as a tuple of words.
        num_words (int): The maximum number of words to generate.
        
    Returns:
        A list of generated words.
    """
    generated_words = list(start_prefix)
    current_prefix = start_prefix

    for _ in range(num_words):
        if current_prefix not in model or not model[current_prefix]:
            break  # Stop if no continuation is found or the model doesn't have the current prefix
        
        # Select the next word based on the distribution of following words
        next_words = model[current_prefix]
        choices, weights = zip(*next_words.items())
        next_word = random.choices(choices, weights=weights, k=1)[0]
        
        # Update the prefix and generated words
        generated_words.append(next_word)
        current_prefix = (*current_prefix[1:], next_word)  # Update prefix by removing the first word and adding the new one
    
    return ' '.join(generated_words)

# Example usage
BOS, EOS = '<BOS>', '<EOS>'
# Assuming `model` is your n-gram model built from the text
# start_prefix = (BOS, BOS)  # For generating new text from the beginning
# Or for user input:
user_input = ('My', 'house',)
generated_text = generate_from_ngram(model, user_input, num_words=500)
print(generated_text)


My house is on the country . " You are not a dozen of them in the passage with as little as if they were papers that he shall go mad if I do n't know when you sit on the day when this fellow . He kept the key in the opalescent London reek . Holmes looked thoroughly surprised at my watch . " A sheath - knife . <EOS> <EOS>
